# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), 2018년 5월 23일**

### 출처

* https://github.com/JuliaComputing/JuliaBoxTutorials/tree/master/introductory-tutorials/broader-topics-and-ecosystem/intro-to-julia-DataFrames

### 함께보기

* https://deepstat.tistory.com/69 (01. constructors)(in English)
* https://deepstat.tistory.com/70 (01. constructors)(한글)
* https://deepstat.tistory.com/71 (02. basicinfo)(in English)
* https://deepstat.tistory.com/72 (02. basicinfo)(한글)
* https://deepstat.tistory.com/73 (03. missingvalues)(in English)
* https://deepstat.tistory.com/74 (03. missingvalues)(한글)
* https://deepstat.tistory.com/75 (04. loadsave)(in English)
* https://deepstat.tistory.com/76 (04. loadsave)(한글)

In [1]:
using DataFrames # load package

## 데이터프레임 저장하기 및 불러오기

우리는 패키지의 모든 기능을 다루지는 않는다. 모든 것을 배우려면 문서(documentation)를 참조하는게 좋다.

여기서는 CSV 파일을 읽고 쓰는 `CSV`와 Julia 고유의 바이너리 형식으로 작업 할 수 있는 `JLD`를 로드한다.
(`JLD`는 현재 v1.0.1에서 작동하지 않는다.) 

In [2]:
using CSV
# using JLD

테스트를 위해서 간단한 `데이터프레임`을 만들자.

In [3]:
x = DataFrame(A=[true, false, true], B=[1, 2, missing],
              C=[missing, "b", "c"], D=['a', missing, 'c'])


,A,B,C,D
,Bool,Int64⍰,String⍰,Char⍰
1,true,1,missing,'a'
2,false,2,b,missing
3,true,missing,c,'c'


그리고 `eltypes`를 이용해서 열타입을 보자.

In [4]:
eltypes(x)

4-element Array{Type,1}:
 Bool                  
 Union{Missing, Int64} 
 Union{Missing, String}
 Union{Missing, Char}  

이제 `CSV`를 이용해서 `x`를 디스크에 저장하자. 작업공간(working directory)에 `x.csv`라는 파일 명이 겹치지 않는지 반드시 확인 해야한다.

In [5]:
CSV.write("x.csv", x)

"x.csv"

이제 `x.csv`를 읽어서 어떻게 저장돼있는지 보자.

In [6]:
print(read("x.csv", String))

A,B,C,D
true,1,,a
false,2,b,
true,,c,c


우리는 이를 다시 불러올 수 있다. `use_mmap=false`는 메모리 매핑(memory mapping)을 하지 않도록 한다.

In [7]:
y = CSV.read("x.csv", use_mmap=false)

,A,B,C,D
,Bool⍰,Int64⍰,String⍰,String⍰
1,true,1,missing,a
2,false,2,b,missing
3,true,missing,c,c


`CSV`로 `데이터프레임`을 읽어오면, 기본적으로 모든 열이 `Missing`을 허용하게 된다. 아래를 통해 열 타입이 바뀌었음을 알 수 있다.

In [8]:
eltypes(y)

4-element Array{Type,1}:
 Union{Missing, Bool}  
 Union{Missing, Int64} 
 Union{Missing, String}
 Union{Missing, String}

이번에는 `x`를 바이너리 형태(binary format)으로 저장해보자. 작업공간에 `x.jld`라는 파일 명이 겹치지 않는지 반드시 확인 해야한다.

In [ ]:
save("x.jld", "x", x)

`x.jld`를 `y`로 불러온 후 `x`와 `y`가 같은지 보자.

In [ ]:
y = load("x.jld", "x")

`y`의 열 타입은 `x`와 같은 것을 볼 수 있다.

In [ ]:
eltypes(y)

다음으로, `bigdf.csv`파일과 `bigdf.jld`파일을 만들건데, 마찬가지로 기존에 디스크에 파일이 없어야 한다.

특히, 10^3개의 행과 10^5개의 열이 있는 큰 `데이터프레임`을 디스크에 쓰도록 해서 시간을 확인할거다. *JLD가 더 빠를 거라는 것을 예상할 수 있다.* `compress=true`를 써서 파일 사이즈를 줄이자.

In [ ]:
bigdf = DataFrame(Bool, 10^3, 10^2)
@time CSV.write("bigdf.csv", bigdf)
@time save("bigdf.jld", "bigdf", bigdf)
getfield.(stat.(["bigdf.csv", "bigdf.jld"]), :size)

마지막으로, 다음번 돌릴 때를 위해서 파일을 지우자.

In [9]:
#foreach(rm, ["x.csv", "x.jld", "bigdf.csv", "bigdf.jld"])
rm("x.csv")